## Correlations between fire incidents and meteorological data

<u>**Index:**</u>
1. [Cleaning the data](#1)
2. [Studying the possible correlation of the Fire Incidents at Alt Emporda with the Pluvimetrics and the Temperature ](#2)

    2.1. [Pluvimetrics](#2.1)
    
    2.3. [Temperature](#2.2)
    
    2.4.[Studying the significance of the correlations](#2.3)
    
       
3. [Studying the variability of incidents between seasons](#3) 


In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Arranging the data <a class="anchor" id="1"></a>

In [2]:
data_dir='/Users/belenmontenegro/Documents/Master/AVDM/AVDM_ioib/dades/'

data_csv = pd.read_csv(data_dir + "Dades_d_incidents_operatius_gestionats_pel_CAT112_20231110.csv")


FileNotFoundError: [Errno 2] No such file or directory: '/Users/belenmontenegro/Documents/Master/AVDM/AVDM_ioib/dades/Dades_d_incidents_operatius_gestionats_pel_CAT112_20231110.csv'

In [ ]:
data_csv.pop("PROVINCIA")
data_csv.pop("MUNICIPI")

In [ ]:
data_csv=data_csv[(data_csv["ANY"] <= 2022) & (data_csv["ANY"] >= 2018)]

In [ ]:
data_csv=data_csv.sort_index(axis=0, ascending=False)


In [ ]:
data_csv['TIPUS'].unique()

In [ ]:
delete_values = ['Trànsit', 'Seguretat', 'Altres incidències','Medi ambient','Fuita (aigua, gas, altres)','Civisme']

data_csv=data_csv[~data_csv['TIPUS'].isin(delete_values)]

In [ ]:
data_csv['DATE'] = pd.to_datetime(data_csv['ANY'].astype(str) + '-' + data_csv['MES'].astype(str),format='%Y-%m')

## Studying the possible correlation of the Fire Incidents at Alt Emporda with the Pluvimetrics and the Temperature <a class="anchor" id="2"></a>

### Pluvimetrics <a class="anchor" id="2.1"></a>

In [ ]:
Incendios=data_csv[data_csv['TIPUS']=='Incendi']

Inc_Alt=Incendios[Incendios['COMARCA']=='ALT EMPORDA']

Inc_Alt=pd.DataFrame(Inc_Alt.groupby(["DATE"])['INCIDENTS'].sum())
Inc_Alt=Inc_Alt.reset_index()

pluvi_Alt=pd.read_csv('/Users/belenmontenegro/Documents/Master/AVDM/AVDM_ioib/dades/Clime_data/pluvi_AltEmporda.csv',delimiter=';')
pluvi_Alt['DATE']=pd.to_datetime(pluvi_Alt['DATE'],format='%Y-%m')

df_corr=pd.merge(Inc_Alt, pluvi_Alt, on='DATE')

df_corr = df_corr.rename(columns={'INCIDENTS': 'Fire Incidents'})
df_corr = df_corr.rename(columns={'Pluvi': 'Pluvi (mm)'})


df_corr['Pluvi (mm) (lag=1 month)'] = df_corr['Pluvi (mm)'].shift(1)
df_corr['Pluvi (mm) (lag=2 month)'] = df_corr['Pluvi (mm)'].shift(2)
df_corr['Pluvi (mm) (lag=3 month)'] = df_corr['Pluvi (mm)'].shift(3)
df_corr['Pluvi (mm) (lag=4 month)'] = df_corr['Pluvi (mm)'].shift(4)
df_corr['Pluvi (mm) (lag=5 month)'] = df_corr['Pluvi (mm)'].shift(5)


df_corr = df_corr.drop(range(5))



In [ ]:
from scipy.stats import pearsonr
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(12, 10))

palette = sns.color_palette("Set2")

size=14

# Función para agregar la leyenda con el coeficiente de correlación
def add_correlation_legend(x, y, ax):
    # Calcular el coeficiente de correlación
    corr_coef, _ = pearsonr(x, y)
    
    # Agregar la leyenda con el coeficiente de correlación
    legend_label = f'Corr: {corr_coef:.2f}'
    ax.legend([legend_label],fontsize=14)

# Utilizar la función regplot y agregar la leyenda con el coeficiente de correlación
sns.regplot(x="Pluvi (mm)", y="Fire Incidents", data=df_corr, ax=ax1, color=palette[0])
ax1.set_xlabel("Pluvi (mm)",fontsize=size)
ax1.set_ylabel("Fire Incidents",fontsize=size)
add_correlation_legend(df_corr["Pluvi (mm)"], df_corr["Fire Incidents"], ax1)

sns.regplot(x="Pluvi (mm) (lag=1 month)", y="Fire Incidents", data=df_corr, ax=ax2, color=palette[1])
ax2.set_xlabel("Pluvi (mm) (lag=1 month)",fontsize=size)
ax2.set_ylabel("Fire Incidents",fontsize=size)
add_correlation_legend(df_corr["Pluvi (mm) (lag=1 month)"], df_corr["Fire Incidents"], ax2)

sns.regplot(x="Pluvi (mm) (lag=2 month)", y="Fire Incidents", data=df_corr, ax=ax3, color=palette[2])
ax3.set_xlabel("Pluvi (mm) (lag=2 month)",fontsize=size)
ax3.set_ylabel("Fire Incidents",fontsize=size)
add_correlation_legend(df_corr["Pluvi (mm) (lag=2 month)"], df_corr["Fire Incidents"], ax3)

sns.regplot(x="Pluvi (mm) (lag=3 month)", y="Fire Incidents", data=df_corr, ax=ax4, color=palette[3])
ax4.set_xlabel("Pluvi (mm) (lag=3 month)",fontsize=size)
ax4.set_ylabel("Fire Incidents",fontsize=size)
add_correlation_legend(df_corr["Pluvi (mm) (lag=3 month)"], df_corr["Fire Incidents"], ax4)

sns.regplot(x="Pluvi (mm) (lag=4 month)", y="Fire Incidents", data=df_corr, ax=ax5, color=palette[4])
ax5.set_xlabel("Pluvi (mm) (lag=4 month)",fontsize=size)
ax5.set_ylabel("Fire Incidents",fontsize=size)
add_correlation_legend(df_corr["Pluvi (mm) (lag=4 month)"], df_corr["Fire Incidents"], ax5)

sns.regplot(x="Pluvi (mm) (lag=5 month)", y="Fire Incidents", data=df_corr, ax=ax6, color=palette[5])
ax6.set_xlabel("Pluvi (mm) (lag=5 month)",fontsize=size)
ax6.set_ylabel("Fire Incidents",fontsize=size)
add_correlation_legend(df_corr["Pluvi (mm) (lag=5 month)"], df_corr["Fire Incidents"], ax6)

# Show the plots
#plt.suptitle('Correlation of the fire Incidents with pluvimetrics', fontsize=16)
plt.tight_layout()
plt.savefig('/Users/belenmontenegro/Documents/Master/AVDM/AVDM_ioib/Belen/Analysis/Correlations_pluvi.pdf')
plt.show()

In [ ]:
df_corr_pluvi=df_corr.copy()

### Temperature <a class="anchor" id="2.2"></a>

In [ ]:
Tmax_Alt=pd.read_csv('/Users/belenmontenegro/Documents/Master/AVDM/AVDM_ioib/dades/Clime_data/Tmax_AltEmporda.csv',delimiter=';')
Tmax_Alt['DATE']=pd.to_datetime(Tmax_Alt['DATE'],format='%Y-%m')
Tmin_Alt=pd.read_csv('/Users/belenmontenegro/Documents/Master/AVDM/AVDM_ioib/dades/Clime_data/Tmin_AltEmporda.csv',delimiter=';')
Tmin_Alt['DATE']=pd.to_datetime(Tmin_Alt['DATE'],format='%Y-%m')
Tmean=pd.merge(Tmax_Alt, Tmin_Alt, on='DATE')

Tmean['T'] = Tmean[['Temp', 'Temp2']].mean(axis=1)
Tmean = Tmean.drop(['Temp', 'Temp2'], axis=1)

df_corr=pd.merge(Inc_Alt, Tmean, on='DATE')

df_corr = df_corr.rename(columns={'INCIDENTS': 'Fire Incidents'})
df_corr = df_corr.rename(columns={'T': 'T (\u2103)'})

df_corr['T (\u2103) (lag=1 month)'] = df_corr['T (\u2103)'].shift(1)
df_corr['T (\u2103) (lag=2 month)'] = df_corr['T (\u2103)'].shift(2)
df_corr['T (\u2103) (lag=3 month)'] = df_corr['T (\u2103)'].shift(3)
df_corr['T (\u2103) (lag=4 month)'] = df_corr['T (\u2103)'].shift(4)
df_corr['T (\u2103) (lag=5 month)'] = df_corr['T (\u2103)'].shift(5)


df_corr = df_corr.drop(range(5))

df_corr = df_corr.reset_index(drop=True)

In [ ]:
from scipy.stats import pearsonr
fig, ((ax1, ax2, ax3), (ax4, ax5, ax6)) = plt.subplots(2, 3, figsize=(12, 10))

palette = sns.color_palette("Set2")

size=14

# Función para agregar la leyenda con el coeficiente de correlación
def add_correlation_legend(x, y, ax,location):
    # Calcular el coeficiente de correlación
    corr_coef, _ = pearsonr(x, y)
    
    # Agregar la leyenda con el coeficiente de correlación
    legend_label = f'Corr: {corr_coef:.2f}'
    ax.legend([legend_label],fontsize=14,loc=location)

# Utilizar la función regplot y agregar la leyenda con el coeficiente de correlación
sns.regplot(x="T (\u2103)", y="Fire Incidents", data=df_corr, ax=ax1, color=palette[0])
ax1.set_xlabel("T (\u2103)",fontsize=size)
ax1.set_ylabel("Fire Incidents",fontsize=size)
add_correlation_legend(df_corr["T (\u2103)"], df_corr["Fire Incidents"], ax1,'upper left')

sns.regplot(x="T (\u2103) (lag=1 month)", y="Fire Incidents", data=df_corr, ax=ax2, color=palette[1])
ax2.set_xlabel("T (\u2103) (lag=1 month)",fontsize=size)
ax2.set_ylabel("Fire Incidents",fontsize=size)
add_correlation_legend(df_corr["T (\u2103) (lag=1 month)"], df_corr["Fire Incidents"], ax2,'upper left')

sns.regplot(x="T (\u2103) (lag=2 month)", y="Fire Incidents", data=df_corr, ax=ax3, color=palette[2])
ax3.set_xlabel("T (\u2103) (lag=2 month)",fontsize=size)
ax3.set_ylabel("Fire Incidents",fontsize=size)
add_correlation_legend(df_corr["T (\u2103) (lag=2 month)"], df_corr["Fire Incidents"], ax3,'upper left')

sns.regplot(x="T (\u2103) (lag=3 month)", y="Fire Incidents", data=df_corr, ax=ax4, color=palette[3])
ax4.set_xlabel("T (\u2103) (lag=3 month)",fontsize=size)
ax4.set_ylabel("Fire Incidents",fontsize=size)
add_correlation_legend(df_corr["T (\u2103) (lag=3 month)"], df_corr["Fire Incidents"], ax4,'upper right')

sns.regplot(x="T (\u2103) (lag=4 month)", y="Fire Incidents", data=df_corr, ax=ax5, color=palette[4])
ax5.set_xlabel("T (\u2103) (lag=4 month)",fontsize=size)
ax5.set_ylabel("Fire Incidents",fontsize=size)
add_correlation_legend(df_corr["T (\u2103) (lag=4 month)"], df_corr["Fire Incidents"], ax5,'upper right')

sns.regplot(x="T (\u2103) (lag=5 month)", y="Fire Incidents", data=df_corr, ax=ax6, color=palette[5])
ax6.set_xlabel("T (\u2103) (lag=5 month)",fontsize=size)
ax6.set_ylabel("Fire Incidents",fontsize=size)
add_correlation_legend(df_corr["T (\u2103) (lag=5 month)"], df_corr["Fire Incidents"], ax6,'upper right')

# Show the plots
#plt.suptitle('Correlation of the fire Incidents with the mean Temperatures', fontsize=16)
plt.tight_layout()
plt.savefig('/Users/belenmontenegro/Documents/Master/AVDM/AVDM_ioib/Belen/Analysis/Correlations_temp.pdf')
plt.show()

### Studying the significance of the correlations <a class="anchor" id="2.3"></a>

In [ ]:
df_corr_temp=df_corr.copy()

corr_random_temp=[]
corr_random_pluvi=[]

for i in range(1000):  
    values_temp=df_corr_temp['T (\u2103)'].tolist()
    np.random.shuffle(values_temp)
    values_pluvi=df_corr_pluvi['Pluvi (mm)'].tolist()
    np.random.shuffle(values_pluvi)
    y=df_corr_temp["Fire Incidents"].tolist()
    corr_coef, _ = pearsonr(values_temp, y)
    corr_random_temp.append(corr_coef)
    y=df_corr_pluvi["Fire Incidents"].tolist()
    corr_coef, _ = pearsonr(values_pluvi, y)
    corr_random_pluvi.append(corr_coef)



In [ ]:
2*np.std(np.array(corr_random_temp))

In [ ]:
def gaussian(x, mu, sigma):
    return (1.0 / (sigma * np.sqrt(2 * np.pi))) * np.exp(-0.5 * ((x - mu) / sigma)**2)

from scipy.stats import norm

In [ ]:
sigma_p=np.std(np.array(corr_random_pluvi)) ; mu_p=np.mean(np.array(corr_random_pluvi)) 
sigma_t=np.std(np.array(corr_random_temp)) ; mu_t=np.mean(np.array(corr_random_temp)) 
x=np.linspace(-0.4,0.4,100) ; y_p=gaussian(x,mu_p,sigma_p) ; y_t=gaussian(x,mu_t,sigma_t)

plt.figure(figsize=(12,8),dpi=180)

size=15

plt.subplot(1,2,1)
plt.hist(corr_random_pluvi,bins=20,density=True,fill=False,histtype='step',edgecolor='black',label='Histogram')
plt.plot(x,y_p,'b-',label='Gaussian')
arrow_start = (mu_p - 2*sigma_p, norm.pdf(mu_p - 2*sigma_p, mu_p, sigma_p))
arrow_end = (mu_p + 2*sigma_p, norm.pdf(mu_p + 2*sigma_p, mu_p, sigma_p))
plt.annotate('', xy=arrow_end, xytext=arrow_start,
             arrowprops={'arrowstyle':'<->', 'color':'blue'}, annotation_clip=False)

midpoint = ((arrow_start[0] + arrow_end[0]) / 2, (arrow_start[1] + arrow_end[1]) / 2)

plt.text(midpoint[0], midpoint[1] + 0.1, f'Confidence Interval (95 %)', 
         horizontalalignment='center', verticalalignment='center', color='blue',fontsize=13)

plt.xlabel('Random correlation coefficients',fontsize=size)
plt.ylabel('P',fontsize=size)
plt.title('Fire Incidents with Pluvimetrics',fontsize=size)
plt.legend(loc='upper right',fontsize=size)

plt.subplot(1,2,2)
plt.hist(corr_random_temp,bins=20,density=True,fill=False,histtype='step',edgecolor='black',label='Histogram')
plt.plot(x,y_p,'r-',label='Gaussian')
arrow_start = (mu_t - 2*sigma_t, norm.pdf(mu_t - 2*sigma_t, mu_t, sigma_t))
arrow_end = (mu_t + 2*sigma_t, norm.pdf(mu_t + 2*sigma_t, mu_t, sigma_t))
plt.annotate('', xy=arrow_end, xytext=arrow_start,
             arrowprops={'arrowstyle':'<->', 'color':'red'}, annotation_clip=False)

midpoint = ((arrow_start[0] + arrow_end[0]) / 2, (arrow_start[1] + arrow_end[1]) / 2)

plt.text(midpoint[0], midpoint[1] + 0.1, f'Confidence Interval (95 %)', 
         horizontalalignment='center', verticalalignment='center', color='red',fontsize=13)

plt.xlabel('Random correlation coefficients',fontsize=size)
plt.ylabel('P',fontsize=size)
plt.title('Fire Incidents with Temperature',fontsize=size)
plt.legend(loc='upper right',fontsize=size)


plt.tight_layout()


plt.savefig('/Users/belenmontenegro/Documents/Master/AVDM/AVDM_ioib/Belen/Analysis/Significance_corr_temp.pdf')

## Studying the variability of incidents between seasons <a class="anchor" id="3"></a>

In [ ]:

data_autumn=data_csv[(data_csv["MES"] <= 11) & (data_csv["MES"] >= 9)]
data_spring=data_csv[(data_csv["MES"] <= 5) & (data_csv["MES"] >= 3)]
data_summer=data_csv[(data_csv["MES"] <= 8) & (data_csv["MES"] >= 6)]
data_winter=data_csv[(data_csv["MES"] <= 12) & (data_csv["MES"] >= 2)]

In [ ]:
data_autumn=pd.DataFrame(data_autumn.groupby('TIPUS')['INCIDENTS'].sum()).reset_index()

data_spring=pd.DataFrame(data_spring.groupby('TIPUS')['INCIDENTS'].sum()).reset_index()

data_summer=pd.DataFrame(data_summer.groupby('TIPUS')['INCIDENTS'].sum()).reset_index()

data_winter=pd.DataFrame(data_winter.groupby('TIPUS')['INCIDENTS'].sum()).reset_index()


In [ ]:
data_autumn.loc[1,'TIPUS']='Medical Assistance' ; data_winter.loc[1,'TIPUS']='Medical Assistance' ; data_spring.loc[1,'TIPUS']='Medical Assistance' ; data_summer.loc[1,'TIPUS']='Medical Assistance'

data_autumn.loc[2,'TIPUS']='Fire' ;  data_winter.loc[2,'TIPUS']='Fire' ; data_spring.loc[2,'TIPUS']='Fire' ; data_summer.loc[2,'TIPUS']='Fire'

data_autumn.loc[3,'TIPUS']='Meteorology' ; data_winter.loc[3,'TIPUS']='Meteorology' ; data_spring.loc[3,'TIPUS']='Meteorology' ; data_summer.loc[3,'TIPUS']='Meteorology' 

In [ ]:
plt.figure(figsize=(15,13),dpi=180)


plt.subplot(2,2,1)

plt.title('Autumn Incidents',size=18)

explode = (0, 0, 0, 0.1) 

pie=plt.pie(data_autumn['INCIDENTS'],colors=plt.cm.Set1.colors,explode=explode, autopct='%1.1f%%', pctdistance=1.2, textprops={'fontsize': 13})
plt.legend(pie[0],data_autumn['TIPUS'],bbox_to_anchor=(1,0.2),loc='best',fontsize=13)

plt.subplot(2,2,2)

plt.title('Winter Incidents',size=18)

plt.pie(data_winter['INCIDENTS'],colors=plt.cm.Set1.colors, autopct='%1.1f%%', pctdistance=1.2, textprops={'fontsize': 13})

plt.subplot(2,2,3)

plt.title('Spring Incidents',size=18)

plt.pie(data_spring['INCIDENTS'],colors=plt.cm.Set1.colors, autopct='%1.1f%%', pctdistance=1.2, textprops={'fontsize': 13})

plt.subplot(2,2,4)

plt.title('Summer Incidents',size=18)

explode = (0, 0, 0.1, 0) 

plt.pie(data_summer['INCIDENTS'],colors=plt.cm.Set1.colors, explode=explode,autopct='%1.1f%%', pctdistance=1.2, textprops={'fontsize': 13})

plt.tight_layout()

plt.savefig('/Users/belenmontenegro/Documents/Master/AVDM/AVDM_ioib/Belen/Analysis/Incidents_seasons.pdf')